In [1]:
%run ./Preamble.ipynb

all_models = PM.check_for_existing_patterns("activations")
# Uncomment below lines to use specific model sets:
# all_models=PM.gpt2xl_models
# all_models=PM.gpt2_models
# all_models=['gpt2','gpt2-untrained_1','gpt2-untrained_1_weight_config_all']
print(all_models)

## load preprocessed data
# ys = PM.load_ys(v=0) # , use_cache=False)
# Xss = PM.load_Xss(all_models, v=0) # , use_cache=False)
ys_6 = PM.load_ys(compress_to_POS_6=True,v=0) # , use_cache=False)
# Xss_6 = PM.load_Xss(all_models, compress_to_POS_6=True,v=0) # , use_cache=False)


['gpt2-xl', 'gpt2-xl-untrained_1', 'gpt2', 'gpt2-untrained_1', 'gpt2-untrained_2', 'gpt2-untrained_3', 'gpt2-untrained_4', 'gpt2-untrained_5', 'gpt2-untrained_6', 'gpt2-untrained_7', 'gpt2-untrained_8', 'gpt2-untrained_9', 'gpt2-untrained_1_weight_config_all', 'gpt2-untrained_2_weight_config_all', 'gpt2-untrained_3_weight_config_all', 'gpt2-untrained_4_weight_config_all', 'gpt2-untrained_5_weight_config_all', 'gpt2-untrained_6_weight_config_all', 'gpt2-untrained_7_weight_config_all', 'gpt2-untrained_8_weight_config_all', 'gpt2-untrained_9_weight_config_all']


loading mydatadict:   0%|          | 0/22 [00:00<?, ?it/s]

## Load data for analysis

In [2]:
exp = "exp2" # experiment folder code

# Load data frames for feature selectivity analysis
# master_activations_df = pd.read_pickle(PM.directories[exp] / f"master_activations_df_quicksave.pkl")
# rank_sums_df = PM.load_data(exp, f"rank_sums_df_quicksave.csv")
FS_pmax_df = PM.load_data(exp, f"FS_pmax_df_quicksave.csv")  # Load precomputed data for feature selectivity
FS_df = PM.load_data(exp, f"FS_df_quicksave.csv")

# Prepare the data for analysis using custom DataDisplayer utility functions
FS_pmax_df = DD.prep_df_for_analysis(FS_pmax_df, scale_xl_layer=False)
FS_df = DD.prep_df_for_analysis(FS_df, scale_xl_layer=False)

# Rename columns for display purposes
column_renamer = DD.df_display_renameer
save = True  # Set save flag to True for saving the outputs

# Extract selective neurons for all categories in `sels`
selective_neurons_all_categories = [FS_df[FS_df[sel] == 1][["model", "layer_idx", "neuron_idx"]] for sel in sels]

# Display the number of selective neurons per category
for sel, sel_inds in zip(sels, selective_neurons_all_categories):
    print(sel, len(sel_inds))

sel_0 251
sel_1 262
sel_2 2813
sel_3 87645
sel_4 389
sel_5 2122


# Plot

In [3]:
# ================================================
# Plot: Non-XL Models
# ================================================

# Reload LaTeX tools and DataDisplayer to ensure updates are reflected
_=importlib.reload(my_latex_tools)
from my_latex_tools import *
_=importlib.reload(DataDisplayer)
DD = DataDisplayer.DataDisplayer(PM)

# Prepare data for non-XL models
df = DD.prep_df_for_analysis(FS_df, scale_xl_layer=False).copy()
df = df[df["model"].isin(PM.gpt2_models)]  # Filter to only include non-XL GPT-2 models

# Group by model group and layer index, calculate mean selectivity for each feature
grouped_sels_df = df.groupby(["model_group", "layer_idx"], sort=False)[sels].mean()

# Convert values to percentages for display purposes
grouped_sels_df = DD.df_display_values_as_percentages(grouped_sels_df)

# Prepare the data for display (format indexes)
dis_res_df = DD.convert_df_for_display(grouped_sels_df, convert_idx=True)

# Optionally save the result as a LaTeX table
if save:
    save_df_as_tex(dis_res_df, fn="counts_of_feature_selective_neurons", v=0, index=True, escape=True)

# Display the results in a readable format
DD.display_full(dis_res_df)


\begin{table}
    \centering
    \input{../inputs/tbl/counts_of_feature_selective_neurons}
    \caption{caption for tbl:counts of feature selective neurons}
    \label{tbl:counts_of_feature_selective_neurons}
    \end{table} % \ref{tbl:counts_of_feature_selective_neurons}


Noun   Verb Adposition Determiner Adjective Adverb
Model         Layer                                                     
GPT-Trained   0      0.26%  1.56%      6.38%      8.59%     1.17%  1.04%
              1      0.78%   1.3%      3.78%      8.72%     1.95%  2.86%
              2      1.04%   1.3%      4.56%      9.77%     1.82%  3.39%
              3      1.43%  1.17%      4.17%       8.2%     2.47%  2.99%
              4      2.08%   1.3%      3.26%      6.25%     2.34%  3.26%
              5      1.69%  1.69%      2.99%      5.86%      2.6%  2.99%
              6      2.47%  1.56%      3.26%      4.95%     2.86%  2.73%
              7       2.6%  1.95%      3.26%      4.95%     3.26%  3.78%
              8      2.86%  1.95%      3.52%      4.43%     2.21%  3.91%
              9      2.34%  1.04%      4.17%      4.95%     2.47%  3.65%
              10     2.47%   1.3%      5.08%      5.21%     1.95%  3.39%
              11     2.21%  1.04%       5.6%      4.69%     1.04%  2.34%
              12     1.43%  1.17%      5.99%      4.17%     1.04%  3.12%
GPT-Untrained 0         0%  0.14%      4.85%     20.38%     0.06%  0.23%
              1      0.01%  0.13%      3.05%     16.57%     0.04%  0.82%
              2         0%  0.03%      1.88%     14.92%     0.06%  1.07%
              3      0.03%  0.03%      1.43%     13.63%        0%  1.07%
              4         0%  0.04%      1.35%     12.73%     0.03%   1.3%
              5      0.03%  0.04%      1.27%     11.86%        0%  0.94%
              6         0%  0.09%      0.98%     11.39%        0%  0.88%
              7         0%     0%      0.77%     11.24%     0.03%   1.1%
              8         0%  0.03%      0.84%     10.87%     0.03%  0.93%
              9         0%  0.04%      0.84%     10.27%     0.03%  1.01%
              10     0.01%  0.04%      0.84%      9.74%        0%  0.91%
              11     0.01%  0.03%      0.78%      9.62%     0.03%  1.04%
              12     0.01%  0.03%      0.62%      9.26%     0.03%  0.85%
GPT-Guassian  0         0%  0.14%      4.85%     20.38%     0.06%  0.23%
              1         0%     0%       0.2%      7.51%     0.01%  0.27%
              2         0%     0%       0.2%      7.47%     0.01%  0.33%
              3         0%     0%      0.23%      7.48%     0.01%  0.32%
              4         0%     0%       0.2%      7.38%     0.01%  0.33%
              5         0%     0%       0.2%      7.44%     0.01%  0.33%
              6         0%     0%      0.22%      7.31%     0.01%  0.32%
              7         0%     0%      0.23%      7.26%     0.01%  0.32%
              8         0%     0%      0.25%      7.22%     0.01%  0.32%
              9         0%     0%      0.25%      7.35%     0.01%  0.32%
              10        0%     0%      0.25%      7.26%     0.01%  0.29%
              11        0%     0%      0.23%      7.44%     0.01%  0.32%
              12        0%     0%      0.27%      7.41%     0.01%  0.32%

In [4]:
# ================================================
# Plot: XL Models
# ================================================

# Reload LaTeX tools and DataDisplayer to ensure updates are reflected
_=importlib.reload(my_latex_tools)
from my_latex_tools import *
_=importlib.reload(DataDisplayer)
DD = DataDisplayer.DataDisplayer(PM)

# Prepare data for XL models
df = DD.prep_df_for_analysis(FS_df, scale_xl_layer=False).copy()
df = df[~df["model"].isin(PM.gpt2_models)]  # Filter to only include GPT-2 XL models

# Group by model group and layer index, calculate mean selectivity for each feature
grouped_sels_df = df.groupby(["model_group", "layer_idx"], sort=False)[sels].mean()

# Convert values to percentages for display purposes
grouped_sels_df = DD.df_display_values_as_percentages(grouped_sels_df)

# Prepare the data for display (format indexes)
dis_res_df = DD.convert_df_for_display(grouped_sels_df, convert_idx=True)

# Optionally save the result as a LaTeX table
if save:
    save_df_as_tex(dis_res_df, fn="counts_of_feature_selective_neurons_xl", v=0, index=True, escape=True)

# Display the results in a readable format
DD.display_full(dis_res_df)

\begin{table}
    \centering
    \input{../inputs/tbl/counts_of_feature_selective_neurons_xl}
    \caption{caption for tbl:counts of feature selective neurons xl}
    \label{tbl:counts_of_feature_selective_neurons_xl}
    \end{table} % \ref{tbl:counts_of_feature_selective_neurons_xl}


Noun   Verb Adposition Determiner Adjective Adverb
Model        Layer                                                     
XL-Trained   0      1.31%  1.06%      4.94%     16.25%      1.0%  0.56%
             1      0.19%  0.69%      1.56%     20.56%     1.25%  1.31%
             2      0.12%  0.31%       0.5%     39.75%     0.75%  0.62%
             3      0.19%  0.19%      0.06%     51.31%     0.69%  0.25%
             4      0.12%  0.19%         0%     57.12%     0.69%  0.12%
             5      0.06%  0.12%         0%     63.75%     0.69%     0%
             6      0.06%  0.06%         0%     69.19%      0.5%  0.12%
             7         0%  0.06%         0%     72.94%     0.38%  0.06%
             8         0%  0.06%         0%     75.56%     0.38%  0.06%
             9         0%  0.06%         0%     78.38%     0.38%  0.06%
             10        0%  0.06%         0%     79.44%     0.25%  0.06%
             11        0%     0%         0%     83.44%     0.25%  0.06%
             12        0%     0%         0%     83.56%     0.19%  0.06%
             13        0%     0%         0%     85.75%     0.06%  0.06%
             14        0%     0%         0%      87.5%        0%  0.06%
             15        0%     0%         0%     88.25%        0%  0.06%
             16        0%     0%         0%     88.25%        0%  0.06%
             17        0%     0%         0%     88.75%        0%  0.06%
             18        0%     0%         0%      90.0%        0%  0.06%
             19        0%     0%         0%     90.44%        0%  0.06%
             20        0%     0%         0%     89.69%        0%  0.06%
             21        0%     0%         0%     89.56%        0%  0.06%
             22        0%     0%         0%     90.19%        0%  0.06%
             23        0%     0%         0%      90.0%        0%  0.06%
             24        0%     0%         0%     90.62%        0%  0.06%
             25        0%     0%         0%     90.62%        0%  0.12%
             26        0%     0%         0%     91.38%        0%  0.06%
             27        0%     0%         0%     91.94%        0%  0.06%
             28        0%     0%         0%     92.94%        0%     0%
             29        0%     0%         0%      94.0%        0%     0%
             30        0%     0%         0%     95.06%        0%     0%
             31        0%     0%         0%     96.38%        0%     0%
             32        0%     0%         0%     97.12%        0%     0%
             33        0%     0%         0%     97.56%        0%     0%
             34        0%     0%         0%     97.88%        0%     0%
             35        0%     0%         0%     98.12%        0%     0%
             36        0%     0%         0%     97.94%        0%     0%
             37        0%     0%         0%     98.19%        0%     0%
             38        0%     0%         0%      98.5%        0%     0%
             39        0%     0%         0%     98.12%        0%     0%
             40        0%     0%         0%     98.12%        0%     0%
             41        0%     0%         0%     98.06%        0%     0%
             42        0%     0%         0%     97.88%     0.06%     0%
             43        0%     0%         0%     97.62%     0.06%     0%
             44        0%     0%         0%     97.56%     0.06%     0%
             45        0%     0%         0%     96.94%     0.06%  0.06%
             46        0%     0%         0%     96.69%     0.06%  0.06%
             47        0%     0%         0%     96.12%     0.06%  0.06%
             48     1.75%  0.94%      6.56%      3.69%     0.75%  2.94%
XL-Untrained 0         0%  0.12%      4.81%     20.19%        0%  0.31%
             1         0%     0%      1.88%     10.75%        0%  1.06%
             2         0%     0%      0.75%      8.38%        0%  1.25%
             3         0%     0%      0.81%      7.31%        0%   0.5%
             4         0%     0%      0.75%       6.0%       

In [5]:
# ================================================
# Plot Final Results
# ================================================

# Reload LaTeX tools and DataDisplayer to ensure updates are reflected
_=importlib.reload(my_latex_tools)
from my_latex_tools import *
_=importlib.reload(DataDisplayer)
DD = DataDisplayer.DataDisplayer(PM)

# Group by model and layer index, calculate mean selectivity for each feature
grouped_sels_df = FS_df.groupby(["model", "layer_idx"])[sels].mean()

# Convert values to percentages for display purposes
res_df = DD.df_display_values_as_percentages(grouped_sels_df)

# Prepare the data for display (format indexes)
dis_res_df = DD.convert_df_for_display(res_df, convert_idx=True)

# Optionally save the result as a LaTeX table
if save:
    save_df_as_tex(dis_res_df, fn="counts_of_feature_selective_neurons", v=1, index=True, escape=True)

# Display the final results in a readable format
DD.display_full(dis_res_df)


\begin{table}
    \centering
    \input{../inputs/tbl/counts_of_feature_selective_neurons}
    \caption{caption for tbl:counts of feature selective neurons}
    \label{tbl:counts_of_feature_selective_neurons}
    \end{table} % \ref{tbl:counts_of_feature_selective_neurons}
\begin{tabular}{llllllll}
\toprule
 &  & Noun & Verb & Adposition & Determiner & Adjective & Adverb \\
Model & Layer &  &  &  &  &  &  \\
\midrule
\multirow[t]{13}{*}{GPT-Trained} & 0 & 0.26\% & 1.56\% & 6.38\% & 8.59\% & 1.17\% & 1.04\% \\
 & 1 & 0.78\% & 1.3\% & 3.78\% & 8.72\% & 1.95\% & 2.86\% \\
 & 2 & 1.04\% & 1.3\% & 4.56\% & 9.77\% & 1.82\% & 3.39\% \\
 & 3 & 1.43\% & 1.17\% & 4.17\% & 8.2\% & 2.47\% & 2.99\% \\
 & 4 & 2.08\% & 1.3\% & 3.26\% & 6.25\% & 2.34\% & 3.26\% \\
 & 5 & 1.69\% & 1.69\% & 2.99\% & 5.86\% & 2.6\% & 2.99\% \\
 & 6 & 2.47\% & 1.56\% & 3.26\% & 4.95\% & 2.86\% & 2.73\% \\
 & 7 & 2.6\% & 1.95\% & 3.26\% & 4.95\% & 3.26\% & 3.78\% \\
 & 8 & 2.86\% & 1.95\% & 3.52\% & 4.43\% & 2.21\% & 3.91\%

Noun   Verb Adposition Determiner Adjective Adverb
Model           Layer                                                     
GPT-Trained     0      0.26%  1.56%      6.38%      8.59%     1.17%  1.04%
                1      0.78%   1.3%      3.78%      8.72%     1.95%  2.86%
                2      1.04%   1.3%      4.56%      9.77%     1.82%  3.39%
                3      1.43%  1.17%      4.17%       8.2%     2.47%  2.99%
                4      2.08%   1.3%      3.26%      6.25%     2.34%  3.26%
                5      1.69%  1.69%      2.99%      5.86%      2.6%  2.99%
                6      2.47%  1.56%      3.26%      4.95%     2.86%  2.73%
                7       2.6%  1.95%      3.26%      4.95%     3.26%  3.78%
                8      2.86%  1.95%      3.52%      4.43%     2.21%  3.91%
                9      2.34%  1.04%      4.17%      4.95%     2.47%  3.65%
                10     2.47%   1.3%      5.08%      5.21%     1.95%  3.39%
                11     2.21%  1.04%       5.6%      4.69%     1.04%  2.34%
                12     1.43%  1.17%      5.99%      4.17%     1.04%  3.12%
GPT-Untrained 1 0         0%  0.26%      4.04%      22.4%     0.13%  0.39%
                1         0%     0%      3.39%     16.02%     0.13%  0.65%
                2         0%     0%      2.08%     14.06%        0%  0.26%
                3      0.13%  0.13%      1.17%      15.1%        0%  0.39%
                4         0%     0%      1.43%     14.32%        0%  0.65%
                5      0.26%     0%      1.04%     11.46%        0%  0.26%
                6         0%  0.13%      1.17%     11.85%        0%  0.78%
                7         0%     0%      0.65%     11.98%        0%  0.65%
                8         0%     0%      0.65%     11.72%        0%  0.52%
                9         0%     0%      0.39%     10.16%        0%  0.52%
                10        0%     0%      0.78%      9.38%        0%  0.39%
                11        0%     0%      0.39%      9.11%        0%  0.52%
                12        0%     0%      0.52%      8.33%        0%  0.65%
GPT-Guassian 1  0         0%  0.26%      4.04%      22.4%     0.13%  0.39%
                1         0%     0%      0.26%      8.33%     0.13%  0.13%
                2         0%     0%      0.26%      8.33%     0.13%  0.13%
                3         0%     0%      0.39%      8.33%     0.13%  0.13%
                4         0%     0%      0.26%       8.2%     0.13%  0.13%
                5         0%     0%      0.39%       8.2%     0.13%  0.13%
                6         0%     0%      0.39%      7.94%     0.13%  0.13%
                7         0%     0%      0.39%      8.07%     0.13%  0.13%
                8         0%     0%      0.39%      8.07%     0.13%  0.13%
                9         0%     0%      0.26%      8.33%     0.13%  0.13%
                10        0%     0%      0.26%      8.46%     0.13%  0.13%
                11        0%     0%      0.26%      8.59%     0.13%  0.13%
                12        0%     0%      0.52%      8.07%     0.13%  0.26%
GPT-Untrained 2 0         0%  0.13%      4.82%     20.96%        0%  0.13%
                1         0%  0.13%      1.82%     18.23%        0%  1.04%
                2         0%     0%      1.69%     15.76%        0%   1.3%
                3         0%     0%      0.91%     12.76%        0%  1.69%
                4         0%     0%      1.43%     11.98%        0%  1.56%
                5         0%  0.13%       1.3%     10.68%        0%  1.17%
                6         0%  0.13%      0.52%     11.07%        0%  1.17%
                7         0%     0%      0.78%     11.46%        0%  1.43%
                8         0%     0%      0.78%     10.81%        0%  1.17%
                9         0%     0%      0.78%     10.03%        0%  1.56%
                10        0%     0%      0.52%      9.38%        0%  1.43%
                11        0%     0%      0.65%      8.59%        0%  1.69%
                12        0%     0%      0.52%   